<a href="https://colab.research.google.com/github/robson-rsp/datascience/blob/main/clustering/content_based_book_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install ipython-autotime  --upgrade
%pip install matplotlib        --upgrade
%pip install pandas            --upgrade
%pip install scikit-learn      --upgrade
%pip install seaborn           --upgrade

'%pip install ipython-autotime  --upgrade\n%pip install matplotlib        --upgrade\n%pip install pandas            --upgrade\n%pip install scikit-learn      --upgrade\n%pip install seaborn           --upgrade'

In [ ]:
from google.colab import drive, files
import pandas as pd
import warnings

drive.mount('/content/drive', force_remount=True)
warnings.filterwarnings("ignore")
%load_ext autotime

Mounted at /content/drive
time: 720 µs (started: 2023-04-18 23:54:11 +00:00)


In [ ]:
links   = pd.read_csv('/content/drive/MyDrive/datasets/movielens-movies/links.csv', encoding='utf-8')
movies  = pd.read_csv('/content/drive/MyDrive/datasets/movielens-movies/movies.csv', encoding='utf-8')
ratings = pd.read_csv('/content/drive/MyDrive/datasets/movielens-movies/ratings.csv', encoding='utf-8')
tags    = pd.read_csv('/content/drive/MyDrive/datasets/movielens-movies/tags.csv', encoding='utf-8')

time: 202 ms (started: 2023-04-18 23:54:30 +00:00)


# EDA

## Links dataframe

In [ ]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


time: 24.4 ms (started: 2023-04-18 23:54:11 +00:00)


In [ ]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB
time: 18.8 ms (started: 2023-04-18 23:54:12 +00:00)


Verificando quais atributos têm valores NaN.

In [ ]:
for col in links.columns:
  print(f'{col}: {links[col].hasnans}')

movieId: False
imdbId: False
tmdbId: True
time: 7.56 ms (started: 2023-04-18 23:54:12 +00:00)


## Movies dataframe

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


time: 19.4 ms (started: 2023-04-18 23:54:12 +00:00)


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
time: 33.3 ms (started: 2023-04-18 23:54:12 +00:00)


Verificando quais atributos têm valores NaN.

In [ ]:
for col in movies.columns:
  print(f'{col}: {movies[col].hasnans}')

movieId: False
title: False
genres: False
time: 22.1 ms (started: 2023-04-18 23:54:12 +00:00)


## Ratings dataframe

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


time: 42.9 ms (started: 2023-04-18 23:54:12 +00:00)


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
time: 37 ms (started: 2023-04-18 23:54:12 +00:00)


Verificando quais atributos têm valores NaN.

In [ ]:
for col in ratings.columns:
  print(f'{col}: {ratings[col].hasnans}')

userId: False
movieId: False
rating: False
timestamp: False
time: 11.4 ms (started: 2023-04-18 23:54:12 +00:00)


## Tags dataframe

In [ ]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


time: 23 ms (started: 2023-04-18 23:54:12 +00:00)


In [ ]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB
time: 11.7 ms (started: 2023-04-18 23:54:12 +00:00)


Verificando quais atributos têm valores NaN.

In [ ]:
for col in tags.columns:
  print(f'{col}: {tags[col].hasnans}')

userId: False
movieId: False
tag: False
timestamp: False
time: 8.8 ms (started: 2023-04-18 23:54:12 +00:00)


## Ratings e Tag dataframes


Vou checar a quantidade de usuários diferentes em 'ratings' e 'tag'.

In [ ]:
print(len(ratings['userId'].unique()))
print(len(tags['userId'].unique()))

610
58
time: 5.01 ms (started: 2023-04-18 23:54:12 +00:00)


Pelo visto nem todos os usuários aplicaram tags aos filmes que assistiram, mas não acredito que isso será um problema.

# Train/Test split

Como a tarefa não é supervisionada, mas sim de agrupamento, não há necessidade de se fazer essa separação.

# Feature engineering

As transformações que farei nos conjuntos de dados são tão simples que não vou usar classes transformadoras nem pipelines neste projeto.

A principal ideia é formar um **conjunto de dados de filmes** onde os gêneros e as tags serão as colunas, e cada filme, uma linha. Essa matriz será usada para formar o **vetor de perfil do usuário** a partir dos filmes que ele já assistiu. Esse vetor será usado para estipular a nota de todos os outros filmes que o usuário ainda não assistiu. Com essas prováveis avaliações, basta ordenarmos de forma decrescente os filmes em relação as suas notas e recomendarmos um número x de filmes.

In [ ]:
from nltk.corpus   import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
import numpy as np

nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

time: 1.63 s (started: 2023-04-18 23:54:12 +00:00)


**Etapa ##:** Formar um dataset raíz, apenas com os atributos necessários para formar o vetor de perfil de usuário e a matriz de filmes.

In [ ]:
ratings_tags = ratings.merge(tags, on='userId', how='left')
ratings_tags = ratings_tags.drop(['movieId_y', 'timestamp_x', 'timestamp_y'], axis=1)
ratings_tags = ratings_tags.rename(columns={'movieId_x':'movieId'})

time: 2.61 s (started: 2023-04-18 23:54:35 +00:00)


In [ ]:
dataset = movies.merge(ratings_tags, on='movieId')

time: 1.37 s (started: 2023-04-18 23:54:37 +00:00)


In [ ]:
dataset.head()

,movieId,title,genres,userId,rating,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,NaN
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,NaN
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,way too long
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,NaN
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,NaN


time: 23.7 ms (started: 2023-04-18 23:54:39 +00:00)


**Etapa ##:** Imputar valores nos campos NaN do atributo 'tag'.

In [ ]:
dataset['tag'].fillna('', inplace=True)

time: 284 ms (started: 2023-04-18 23:54:43 +00:00)


**Etapa ##:** Agrupar todas as tags dadas a um filme.

In [ ]:
matrix_movies = dataset.groupby(['movieId', 'genres'], as_index=False).agg({'tag':' '.join})
matrix_movies.set_index('movieId', inplace=True)
matrix_movies.head()

,genres,tag
movieId,,
1,Adventure|Animation|Children|Comedy|Fantasy,way too long Al Pacino gangster mafia Al P...
2,Adventure|Children|Fantasy,Al Pacino gangster mafia Al Pacino Mafia hol...
3,Comedy|Romance,big wave surfing ummarti2...
4,Comedy|Drama|Romance,gothic
5,Comedy,EPIC great soundtrack good dialogue gr...


time: 6.36 s (started: 2023-04-18 23:54:18 +00:00)


**Etapa ##:** Unir todas tags aos gêneros formando uma única coluna que vou chamar de 'tags'. Aqui, também faço alguns ajustes finais, como:

*   Transformar todas as strings em letras minúsculas
*   Substituir o '|' por um espaço em branco
*   Formatar os espaços em branco


In [ ]:
matrix_movies['genres'] = matrix_movies['genres'].str.replace('|', ' ')
matrix_movies['tags']   = matrix_movies['genres'] + matrix_movies['tag']
matrix_movies['tags']   = matrix_movies['tags'].str.lower()
matrix_movies['tags']   = matrix_movies['tags'].str.replace(r'\s{2,}', ' ', regex=True)

matrix_movies = matrix_movies.drop(['genres', 'tag'], axis=1)
matrix_movies.head()

,tags
movieId,
1,adventure animation children comedy fantasy wa...
2,adventure children fantasy al pacino gangster ...
3,comedy romance big wave surfing ummarti2006 no...
4,comedy drama romance gothic
5,comedy epic great soundtrack good dialogue gre...


time: 2.6 s (started: 2023-04-18 23:54:24 +00:00)


**Etapa ##:** Remover stopwords e palavras repetidas.

In [ ]:
def filter_tag(tag):
  words = word_tokenize(tag)
  words = set(words)
  stop_words = set(stopwords.words("english"))
  filtered_sentence = [w for w in words if w not in stop_words]
  return ' '.join(filtered_sentence)



matrix_movies['tags'] = np.vectorize(filter_tag)(matrix_movies['tags'])

time: 977 µs (started: 2023-04-19 00:07:06 +00:00)


In [ ]:
matrix_movies.head()

,tags
movieId,
1,hemingway surreal future peter friend dick ame...
2,hemingway surreal future peter friend dick sce...
3,surreal future dumb friend dick bromantic hole...
4,gothic romance drama comedy
5,hemingway coulda native road biopic future pet...


time: 35.4 ms (started: 2023-04-19 00:08:18 +00:00)
